In [3]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv("data/consumer_complaints.csv")

/var/folders/tm/cclhk5g57cb0tkysh8pv5mvr0000gn/T/ipykernel_50588/927148996.py:1: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("consumer_complaints.csv")


### 1 Change data type

In [6]:
df['date_received'] = pd.to_datetime(df.date_received)
df['date_sent_to_company'] = pd.to_datetime(df['date_sent_to_company'])

In [7]:
df.dtypes

date_received                   datetime64[ns]
product                                 object
sub_product                             object
issue                                   object
sub_issue                               object
consumer_complaint_narrative            object
company_public_response                 object
company                                 object
state                                   object
zipcode                                 object
tags                                    object
consumer_consent_provided               object
submitted_via                           object
date_sent_to_company            datetime64[ns]
company_response_to_consumer            object
timely_response                         object
consumer_disputed?                      object
complaint_id                             int64
dtype: object

### 2 Impute missing values

In [8]:
df.isna().sum()

date_received                        0
product                              0
sub_product                     158322
issue                                0
sub_issue                       343335
consumer_complaint_narrative    489151
company_public_response         470833
company                              0
state                             4887
zipcode                           4505
tags                            477998
consumer_consent_provided       432499
submitted_via                        0
date_sent_to_company                 0
company_response_to_consumer         0
timely_response                      0
consumer_disputed?                   0
complaint_id                         0
dtype: int64

In [12]:
df.fillna({
    'sub_product': 'Unknown',
    'sub_issue': 'Unknown',
    'consumer_complaint_narrative': 'No Narrative',
    'company_public_response': 'No Response',
    'state': 'Unknown',
    'zipcode': 'Unknown',
    'tags': 'No Tags',
    'consumer_consent_provided': 'No Consent Provided'
}, inplace=True)

### 3 Feature Engineering

In [13]:
df['year_month'] = df['date_received'].dt.strftime('%Y-%m')

### 4 Save results

In [15]:
df.to_csv("data/complaints_transformed.csv")